<a href="https://colab.research.google.com/github/uhdang/mlim_group_3_final_assignment/blob/main/project/train_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:

- Downsample training set (reduce method to delete irrelevant rows)
- maybe PCA()

# Google Colab Runtime
Run if using Google Colab

In [1]:
# # # import pandas as pd
# # # 
# # import numpy as np# 
# # 
# import o# s# 
# 
# from google.colab import dri# v# e# 
# 
# import # s# y# s#
# drive.mount('/content/dri# v# e# '# )
# sys.path.append('/content/drive/My Drive/My Dri# v# e# '# )
#
# from dataloader import Dat# a# l# o# ader
# # L# o# a# d#  Data
# path = "/content/drive/MyDrive/My Drive/"

# Local Runtime
Run if using notebook locally

In [24]:
import pandas as pd
import os
from tqdm import tqdm

pd.options.mode.chained_assignment = None  # default='warn'

from dataloader import Dataloader
from dataloader import create_combined_dict

# Load Data
path = os.getcwd() + "/../data/"

# Load Data

In [25]:
# load data which creates baskets and coupons
data = Dataloader(path)

In [26]:
# Create Categories before CV as they don't change and consume time otherwise
data.create_category_table(10000)

In [27]:
%%capture
weeks = [86, 87, 88, 89]
shopper_list = list(range(2000))
shopper_chunks = [shopper_list[i:i + 100] for i in range(0, len(shopper_list), 100)]

cv_dict = {
    'X_train': list(),
    'y_train': list(),
    'X_test': list(),
    'y_test': list()
}

In [28]:
for i, week in enumerate(weeks):
  print(f"week: {week}")

  X_train_list = list()
  y_train_list = list()
  X_test_list = list()
  y_test_list = list()

  for idx, shopper in enumerate(tqdm(shopper_chunks)):
      # print(f"shopper_chunk index: {idx}")

      # train-test-split
      data.train_test_split(week, shopper)

      # data add categories
      data.add_categories()

      # create features
      data.create_feature_dict()

      # combine everything
      if i == 0:
          X_train, y_train, X_test, y_test = data.make_featured_data()
          X_train_list.append(X_train)
          y_train_list.append(y_train)
          del X_train, y_train
      else:
          _, _, X_test, y_test = data.make_featured_data()

      X_test_list.append(X_test)
      y_test_list.append(y_test)
      del X_test, y_test

  cv_dict = create_combined_dict(X_train_list, y_train_list, X_test_list, y_test_list, cv_dict)

 20%|██        | 4/20 [09:07<36:29, 136.84s/it]


week: 86
week: 87


KeyboardInterrupt: 

In [ ]:
cv_dict["X_train"][0]["shopper"]

In [ ]:
import pickle

with open('cv_dict.pickle', 'wb') as handle:
    pickle.dump(cv_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Hyperparameter Tuning

In [7]:
%%capture
!pip install optuna

In [ ]:
import optuna
import optuna.integration.lightgbm as lgb
from sklearn.metrics import roc_auc_score

dtrain = lgb.Dataset(cv_dict['X_train'][0], label=cv_dict['y_train'][0])
dval0 = lgb.Dataset(cv_dict['X_test'][0], label=cv_dict['y_test'][0])
dval1 = lgb.Dataset(cv_dict['X_test'][1], label=cv_dict['y_test'][1])
dval2 = lgb.Dataset(cv_dict['X_test'][2], label=cv_dict['y_test'][2])
dval3 = lgb.Dataset(cv_dict['X_test'][3], label=cv_dict['y_test'][3])

params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

lgb_clf = lgb.train(
    params,
    dtrain,
    valid_sets=[dval0, dval1, dval2, dval3],
    verbose_eval=100,
    early_stopping_rounds=100
)

prediction = lgb_clf.predict_proba(cv_dict['X_test'][3], num_iteration=lgb_clf.best_iteration)[:,1]
AUC = roc_auc_score(
    y_true = cv_dict['y_test'][3],
    y_score = prediction
)


best_params = lgb_clf.params
print("Best params:", best_params)
print("  Accuracy = {}".format(AUC))
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

# Prediction and Revenue Optimization

# Playground

In [ ]:
# train-test-split
data.train_test_split(89, list(range(50)))

# data add categories 
data.add_categories()
data.create_feature_dict()
X_train = data._make_full_table(data.baskets_train, data.coupons_train)
X_test = data._make_full_table(data.baskets_test, data.coupons_test)

X_train = data._merge_features(X_train, data.feature_dict)
X_test = data._merge_features(X_test, data.feature_dict)

In [6]:
X_train.head()

,week,shopper,product,price,target,category,discount,coupon,original_price,ratio_product_count,ratio_category_count,reordered_product,reordered_category,coupon_in_same_category,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper
0,0,0,0,688.0,0.0,12,0.0,No,688,0.000000,0.089888,0.0,1.0,Yes,0.722222,0.960000,587.203947,8.539326,54,25
1,0,1,0,688.0,0.0,12,0.0,No,688,0.067416,0.314607,1.0,1.0,Yes,0.830986,1.000000,584.065350,7.393258,71,24
2,0,2,0,688.0,0.0,12,0.0,No,688,0.000000,0.595506,0.0,1.0,No,0.700000,0.954545,571.918919,5.820225,60,22
3,0,3,0,688.0,0.0,12,0.0,No,688,0.000000,0.415730,0.0,1.0,No,0.714286,1.000000,561.846361,8.337079,91,24
4,0,4,0,688.0,0.0,12,0.0,No,688,0.000000,0.089888,0.0,1.0,No,0.764706,1.000000,583.750455,6.168539,34,20
